# Entraînement du modèle CNN pour la reconnaissance du langage des signes

Ce notebook entraîne un modèle CNN capable de reconnaître les lettres et chiffres du langage des signes.
Le modèle sera sauvegardé dans `model/sign_language_cnn.h5` pour être utilisé par l'application Flask.

In [ ]:
# =========================
# 1. INSTALLATION & IMPORTS (robuste)
# =========================

import subprocess
import sys
import importlib

def ensure_package(pkg_spec):
    """Install a package if it's not importable."""
    name = pkg_spec.split('==')[0].split('>=')[0].split('<=')[0]
    try:
        importlib.import_module(name)
    except Exception:
        print(f'Installation de {pkg_spec}...')
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_spec])

# Ensure typing_extensions recent enough for IPython/guarded_eval (avoid kernel crashes)\
# we pin a minimum version but allow pip to choose a compatible one.
ensure_package('typing_extensions>=4.5.0')

# TensorFlow et kagglehub : n'installer que si nécessaire pour éviter des réinstallations fréquentes.
try:
    import tensorflow as tf
except Exception:
    print('TensorFlow non trouvé : installation en cours (cela peut prendre plusieurs minutes)...')
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'tensorflow==2.13.0'])
    import tensorflow as tf

try:
    import kagglehub
except Exception:
    print('kagglehub non trouvé : installation en cours...')
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'kagglehub'])
    import kagglehub

# Imports principaux
import os
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers

print(f'TensorFlow version: {tf.__version__}')
# Keras peut provenir de tensorflow.keras
try:
    print(f'Keras version: {keras.__version__}')
except Exception:
    print('Keras: version indisponible via keras.__version__')

Installation de TensorFlow (cela peut prendre quelques minutes)...


## 2. Configuration de l'environnement et paramètres

Définissez les chemins et hyperparamètres pour l'entraînement.

In [ ]:
# =========================
# 2. CONFIGURATION
# =========================

# Chemins
PROJECT_ROOT = Path(".").resolve()
MODEL_DIR = PROJECT_ROOT / "model"
MODEL_PATH = MODEL_DIR / "sign_language_cnn.h5"

# Créer le dossier model s'il n'existe pas (parents=True pour robustesse)
MODEL_DIR.mkdir(parents=True, exist_ok=True)

# Hyperparamètres
IMG_SIZE = (64, 64)      # Taille des images (64x64 comme dans ton config.py)
BATCH_SIZE = 32
EPOCHS = 15
SEED = 123

print(f"Dossier du modèle: {MODEL_DIR}")
print(f"Chemin du modèle final: {MODEL_PATH}")
print(f"Taille des images: {IMG_SIZE}")
print(f"Nombre d'époques: {EPOCHS}")

## 3. Télécharger le dataset depuis Kaggle

Utilise `kagglehub` pour télécharger le dataset Sign Language de Kaggle.

In [ ]:
# =========================
# 3. TÉLÉCHARGER LE DATASET
# =========================

import kagglehub

print("Téléchargement du dataset Sign Language depuis Kaggle...")
print("(Cela peut prendre plusieurs minutes la première fois)")

dataset_path = kagglehub.dataset_download("harshvardhan21/sign-language-detection-using-images")
dataset_path = Path(dataset_path)
print(f"Dataset téléchargé dans: {dataset_path}")

# Repérer le dossier contenant les images de classes (qui contient des sous-dossiers par classe)
candidates = [dataset_path / 'data', dataset_path]
data_dir = None
for cand in candidates:
    if cand.exists() and cand.is_dir():
        subdirs = [d for d in cand.iterdir() if d.is_dir()]
        if subdirs and any(any(d.glob('*.jpg')) or any(d.glob('*.png')) for d in subdirs):
            data_dir = cand
            break

# Si non trouvé, chercher plus profondément
if data_dir is None:
    for p in dataset_path.rglob('*'):
        if p.is_dir():
            subdirs = [d for d in p.iterdir() if d.is_dir()]
            if len(subdirs) >= 2 and any(any(d.glob('*.jpg')) or any(d.glob('*.png')) for d in subdirs):
                data_dir = p
                break

if data_dir is None:
    raise FileNotFoundError('Impossible de localiser le dossier dimages dans le dataset téléchargé. Vérifiez dataset_path.')

print(f"Dossier des images trouvé: {data_dir}")

# Lister les classes (sous-dossiers)
classes = sorted([d.name for d in data_dir.iterdir() if d.is_dir()])
print(f"Classes trouvées ({len(classes)}): {classes}")

## 4. Créer les datasets d'entraînement et de validation

Charger les images et les séparer en ensembles d'entraînement (80%) et de validation (20%).

In [ ]:
# =========================
# 4. CRÉER LES DATASETS
# =========================

print("Chargement du dataset d'entraînement...")
try:
    train_ds = tf.keras.utils.image_dataset_from_directory(
        str(data_dir),
        validation_split=0.2,
        subset="training",
        seed=SEED,
        image_size=IMG_SIZE,
        batch_size=BATCH_SIZE
    )
    val_ds = tf.keras.utils.image_dataset_from_directory(
        str(data_dir),
        validation_split=0.2,
        subset="validation",
        seed=SEED,
        image_size=IMG_SIZE,
        batch_size=BATCH_SIZE
    )
except Exception as e:
    raise RuntimeError(f'Erreur lors du chargement des images depuis {data_dir}: {e}')

class_names = train_ds.class_names
num_classes = len(class_names)

print(f"Classes trouvées: {class_names}")
print(f"Nombre de classes: {num_classes}")

# Optimiser le pipeline
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

## 5. Augmentation des données

Appliquer des transformations aléatoires (flip, rotation, zoom) pour améliorer la robustesse du modèle.

In [ ]:
# =========================
# 5. AUGMENTATION DE DONNÉES
# =========================

data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

print("Pipeline d'augmentation de données créé.")

## 6. Construire le modèle CNN

Créer un modèle avec 3 blocs Conv2D + MaxPooling, suivi de couches denses.

In [ ]:
# =========================
# 6. CONSTRUIRE LE MODÈLE CNN
# =========================

print("Construction du modèle CNN...")

inputs = keras.Input(shape=(*IMG_SIZE, 3))
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(x)

# Bloc 1
x = layers.Conv2D(32, (3, 3), activation="relu", padding="same")(x)
x = layers.MaxPooling2D((2, 2))(x)

# Bloc 2
x = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(x)
x = layers.MaxPooling2D((2, 2))(x)

# Bloc 3
x = layers.Conv2D(128, (3, 3), activation="relu", padding="same")(x)
x = layers.MaxPooling2D((2, 2))(x)

# Couches denses
x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(256, activation="relu")(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)

model = keras.Model(inputs, outputs)

print("Résumé du modèle:")
model.summary()

## 7. Compiler et entraîner le modèle

Compiler le modèle avec Adam et sparse_categorical_crossentropy, puis l'entraîner.

In [ ]:
# =========================
# 7. COMPILER ET ENTRAÎNER
# =========================

print("Compilation du modèle...")
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

print(f"\nEntraînement en cours ({EPOCHS} époques)...")
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    verbose=1
)

print("\n✅ Entraînement terminé!")

In [ ]:
# =========================
# 8. SAUVEGARDER LE MODÈLE
# =========================

print("\nSauvegarde du modèle...")

# S'assurer que le dossier model existe
MODEL_DIR.mkdir(parents=True, exist_ok=True)

# Sauvegarder le modèle
model.save(str(MODEL_PATH))

print(f"✅ Modèle sauvegardé dans: {MODEL_PATH}")
print(f"Taille du fichier: {MODEL_PATH.stat().st_size / (1024*1024):.2f} MB")

# Vérifier que le fichier existe
if MODEL_PATH.exists():
    print("✅ Fichier vérifié - Le modèle est prêt pour Flask!")
else:
    print("❌ Erreur: Le fichier n'a pas été créé")